In [3]:
import numpy as np
import pandas as pd
from scipy.io import arff
import pandas as pd
import pytempo
import sys

In [4]:
train = arff.loadarff('BeetleFly_TRAIN.arff')
train = pd.DataFrame(train[0])
test = arff.loadarff('BeetleFly_TEST.arff')
test = pd.DataFrame(test[0])
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 513 entries, att1 to target
dtypes: float64(512), object(1)
memory usage: 80.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 513 entries, att1 to target
dtypes: float64(512), object(1)
memory usage: 80.3+ KB
None


In [5]:
np_train=train.iloc[:,:-1].to_numpy()
np_train

array([[1.2467886 , 1.1749374 , 1.0688188 , ..., 1.1264838 , 1.2218016 ,
        1.2866704 ],
       [0.82587635, 0.88131401, 0.81807343, ..., 0.55837279, 0.64643742,
        0.75525556],
       [2.1328304 , 2.029373  , 1.9404839 , ..., 1.9620999 , 2.0634468 ,
        2.1498487 ],
       ...,
       [2.1416023 , 2.1515685 , 2.0351355 , ..., 1.7240577 , 1.885459  ,
        2.0473709 ],
       [1.9203439 , 1.9255989 , 1.8505462 , ..., 1.5577559 , 1.637236  ,
        1.7979197 ],
       [1.4016579 , 1.3684963 , 1.248251  , ..., 1.0372773 , 1.1970256 ,
        1.3175642 ]])

In [6]:
np_train_label=train.iloc[:,-1].to_numpy()
np_train_label=np_train_label.astype(int)
np_train_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
np_test=test.iloc[:,:-1].to_numpy()
np_test

array([[ 1.7400873 ,  1.7331051 ,  1.7091917 , ...,  1.4434654 ,
         1.5501407 ,  1.6625118 ],
       [ 1.6792112 ,  1.7372853 ,  1.8133272 , ...,  1.3570367 ,
         1.4772951 ,  1.5977733 ],
       [ 0.10087896,  0.17041868,  0.26746747, ..., -0.14894719,
        -0.05224583,  0.02401989],
       ...,
       [ 1.556693  ,  1.6197961 ,  1.7035214 , ...,  1.2933423 ,
         1.3686128 ,  1.4785764 ],
       [ 1.6291159 ,  1.5422846 ,  1.4388813 , ...,  1.3809487 ,
         1.4810994 ,  1.6157037 ],
       [ 2.134566  ,  2.1704899 ,  2.1618218 , ...,  1.9251837 ,
         2.0174724 ,  2.0791975 ]])

In [8]:
np_test_label=test.iloc[:,-1].to_numpy()
np_test_label=np_test_label.astype(int)
np_test_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [12]:
# Euclidean
def Euclidean():
    min_ed =sys.maxsize
    min_y = sys.maxsize
    result_ed=[]
    ed=[]
    for i in range(len(np_test)):
        q = np_test[i]
        min_ed =sys.maxsize
        min_y = sys.maxsize
        for j in range(len(np_train)):
            c= np_train[j]
            v = np.linalg.norm(q - c)
            if(v<min_ed):
                min_ed = v
                min_y = np_train_label[j]
        result_ed.append(min_y)
    return result_ed

# DTW
def dtw():
    min_dtw =sys.maxsize
    min_y = sys.maxsize
    ud = pytempo.univariate.distances
    result=[]
    dtw=[]
    for i in range(len(np_test)):
        q = np_test[i]
        min_dtw =sys.maxsize
        min_y = sys.maxsize
        for j in range(len(np_train)):
            c= np_train[j]
            v = ud.dtw(q, c)
            if(v<min_dtw):
                min_dtw = v
                min_y = np_train_label[j]
        result.append(min_y)
        dtw.append(min_dtw)  
    return result
def cdtw(w):
    min_cdtw =sys.maxsize
    min_y = sys.maxsize
    result_cdtw=[]
    cdtw=[]
    L = len(np_test)
    for i in range(len(np_test)):
        q = np_test[i]
        min_cdtw =sys.maxsize
        min_y = sys.maxsize
        for j in range(len(np_train)):
            c= np_train[j]
            v = ud.cdtw(q, c, w*(L-1))
            if(v<min_cdtw):
                min_cdtw = v
                min_y = np_train_label[j]
        result_cdtw.append(min_y)
        cdtw.append(min_cdtw) 
    return result_cdtw

In [13]:
q1=0
q2=0
q3=0
ud = pytempo.univariate.distances
r1 = Euclidean()
r2 = dtw()
r3 = cdtw(7)
for i in range(len(np_test_label)):
    if r1[i]==np_test_label[i]:
        q1=q1+1
    if r2[i]==np_test_label[i]:
        q2=q2+1
    if r3[i]==np_test_label[i]:
        q3=q3+1   
a1 = q1/len(np_test_label)
a2 = q2/len(np_test_label)
a3 = q3/len(np_test_label)
print(f'Ed: {format(1-a1, ".4f")}')
print(f'DTW(Learned w): {format(1-a3, ".4f")}')
print(f'DTW(w=100): {format(1-a2, ".4f")}')

Ed: 0.2500
DTW(Learned w): 0.3000
DTW(w=100): 0.3000
